In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import sys
os.environ['UTILS_PATH'] = r'C:\Users\Mi\Desktop\code\git\python-utils-ak'
os.environ['PROJECT_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat' 
os.environ['SQLITE_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite' 
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

In [3]:
df = pd.read_csv('2020.12.13 boiling_plan.csv')
df['sku'] = df['sku'].apply(cast_sku)
df['boiling'] = df['sku'].apply(lambda sku: sku.boilings[0])
df.sort_values(by='id')
df['packing_speed'] = df['sku'].apply(lambda sku: sku.packing_speed)
df['boiling_form_factor'] = df['sku'].apply(lambda sku:sku.boiling_form_factors[0])

df.at[:2, 'packing_team_id'] = 1
df.at[2:, 'packing_team_id'] = 2

# mark batch
cur_batch = 0
cur_boiling = None
batch_values = []
for i, row in df.iterrows():
    if row['boiling'] != cur_boiling:
        cur_batch += 1
        cur_boiling = row['boiling']
    batch_values.append(cur_batch)
df['batch'] = batch_values

boiling_plan_df = df.copy()
boiling_plan_df.head()

,id,boiling,sku,kg,packing_speed,boiling_form_factor,packing_team_id,batch
0,1.0,<Boiling 9>,<SKU 25>,1000.0,960,<BoilingFormFactor 13>,1.0,1
1,2.0,<Boiling 9>,<SKU 25>,1000.0,960,<BoilingFormFactor 13>,1.0,1
2,3.0,<Boiling 6>,<SKU 23>,192.8,960,<BoilingFormFactor 1>,2.0,2
3,3.0,<Boiling 6>,<SKU 26>,134.7,960,<BoilingFormFactor 1>,2.0,2
4,3.0,<Boiling 6>,<SKU 46>,153.6,960,<BoilingFormFactor 1>,2.0,2


In [4]:
def gen_boiling_plan(boiling_plan_df, id):
    boiling_plan = boiling_plan_df[boiling_plan_df['id'] == id]

    # mark groups
    cur_group = 0
    cur_form_factor = None
    group_values = []
    for i, row in boiling_plan.iterrows():
        if row['boiling_form_factor'] != cur_form_factor:
            cur_group += 1
            cur_form_factor = row['boiling_form_factor']
        group_values.append(cur_group)
    boiling_plan['group'] = group_values
    return boiling_plan

# Create schedule for sample boiling

In [40]:
def make_boiling(boiling_plan):
    boiling_model = boiling_plan.iloc[0]['boiling']
    
    # todo: del
    if boiling_model.boiling_type == 'salt':
        # todo: make properly
        boiling_model.meltings.speed = 850 / 50 * 60

    maker, root, make = new_block_maker(default_push_func=dummy_push)

    # [termizator.time]
    # todo: del
    termizator = db.session.query(Termizator).first()
    termizator.pouring_time = 30
    
    with make('boiling', boiling_model=boiling_model):
        with make('pouring', push_func=add_push):
            with make('first'):
                make('termizator', time_size=termizator.pouring_time)
                make('fermenting', time_size=boiling_model.pourings.pouring_time - termizator.pouring_time)
                make('soldification', time_size=boiling_model.pourings.soldification_time)
                make('cutting', time_size=boiling_model.pourings.cutting_time)
            with make('second'):
                make('pouring_off', time_size=boiling_model.pourings.pouring_off_time)
                make('extra', time_size=boiling_model.pourings.extra_time)

        make('drenator', 
             t=root['boiling']['pouring']['first'].end(),
             time_size=boiling_model.lines.cheddarization_time * 12 * 5, 
             push_func=add_push)

        with make('melting'):
            serving = make('serving', push_func=add_push, time_size=boiling_model.meltings.serving_time).block

            meltings = make('meltings', push_func=add_push, t=serving.length()).block
            coolings = make('coolings', push_func=add_push, t=serving.length()).block

            for group, grp in boiling_plan.groupby('group'):
                if group > 1 and boiling_model.boiling_type == 'water':
                    # non-first group - reconfigure time
                    dummy_push(meltings, Block('melting_configuration', size=1))

                melting_process = Block('melting_process',
                                        time_size=custom_round(grp['kg'].sum() / boiling_model.meltings.speed * 60, 5, 'ceil'),
                                       actual_time_size=grp['kg'].sum() / boiling_model.meltings.speed * 60,
                                       boiling_form_factor=grp.iloc[0]['boiling_form_factor'])

                dummy_push(meltings, melting_process)

                # add cooling
                cooling_process = Block('cooling_process', t=melting_process.props.relative_props['t'])
                cooling_maker, _, cooling_make = new_block_maker(cooling_process, default_push_func=dummy_push)

                with cooling_make('start'):
                    if boiling_model.boiling_type == 'water':
                        cooling_make('cooling', time_size=boiling_model.meltings.first_cooling_time)
                        cooling_make('cooling', time_size=boiling_model.meltings.second_cooling_time)
                    elif boiling_model.boiling_type == 'salt':
                        cooling_make('salting', time_size=boiling_model.meltings.salting_time)

                with cooling_make('finish'):
                    if boiling_model.boiling_type == 'water':
                        cooling_make('cooling', time_size=melting_process.props['time_size'])
                    elif boiling_model.boiling_type == 'salt':
                        cooling_make('salting', time_size=melting_process.props['time_size'])

                add_push(coolings, cooling_process)
                
    return root['boiling']

In [41]:
boilings = []
ids = list(boiling_plan_df['id'].unique())[:2]
for id in ids:
    boiling_plan = gen_boiling_plan(boiling_plan_df, id)
    boiling = make_boiling(boiling_plan)
    print('Boiling', id, boiling.props['boiling_model'].boiling_type)
    print(draw_print(boiling))
    boilings.append(boiling)


Boiling 1.0 water
================================================================================================== boiling 0:[0,98)
=========================== pouring 0:[0,27)
====================== first 0:[0,22)
====== termizator 0:[0,6)
      == fermenting 0:[6,8)
        ======= soldification 0:[8,15)
               ======= cutting 0:[15,22)
                      ===== second 0:[22,27)
                      === pouring_off 0:[22,25)
                         == extra 0:[25,27)
                      ================================================ drenator 0:[22,70)
                                                                      ============================ melting 0:[70,98)
                                                                      ====== serving 0:[70,76)
                                                                            ============== meltings 0:[76,90)
                                                                            ============== melting_p

<ipython-input-4-79e6b335079c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boiling_plan['group'] = group_values
<ipython-input-4-79e6b335079c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boiling_plan['group'] = group_values


# Combine boilings

In [42]:
def boiling_validator(parent, boiling):
    if not parent.children:
        return

    boilings = [node for node in parent.children if node.props['class'] == 'boiling']

    b2 = boiling

    for b1 in reversed(boilings[-6:]): # compare with four previous boilings
        validate_disjoint(b1['pouring'][0]['termizator'], b2['pouring'][0]['termizator'])  # [termizator.basic]

        # cannot make two boilings on same line at the same time
        if b1['pouring'].props['pouring_line'] == b2['pouring'].props['pouring_line']:
            validate_disjoint(b1['pouring'], b2['pouring'])

        if b1.props['boiling_type'] == b2.props['boiling_type']:
            # [melting.disjoint]
            validate_disjoint(b1['melting']['meltings'], b2['melting']['meltings'])

In [43]:
boiling_plan.head()

,id,boiling,sku,kg,packing_speed,boiling_form_factor,packing_team_id,batch,group
1,2.0,<Boiling 9>,<SKU 25>,1000.0,960,<BoilingFormFactor 13>,1.0,1,1


In [44]:
lines_df = pd.DataFrame(index=['water', 'salt'], columns=['iter_props', 'start_time', 'boiling_ids_left', 'latest_boiling'])
lines_df.at['water', 'iter_props'] = [{'pouring_line': str(v)} for v in [0, 1]]
lines_df.at['salt', 'iter_props'] = [{'pouring_line': str(v)} for v in [2, 3]]

# [cheesemakers.start_time]
lines_df.at['water', 'start_time'] = '09:50'
lines_df.at['salt', 'start_time'] = '07:05'

for boiling_type in ['water', 'salt']:
    ids = boiling_plan_df[boiling_plan_df['boiling'].apply(lambda b: b.boiling_type) == boiling_type].id
    ids = list(ids)
    ids = remove_duplicates(ids)
    lines_df.at[boiling_type, 'boiling_ids_left'] = ids
    
lines_df['latest_boiling'] = None
lines_df

,iter_props,start_time,boiling_ids_left,latest_boiling
water,"[{'pouring_line': '0'}, {'pouring_line': '1'}]",09:50,"[1.0, 2.0, 3.0, 4.0, 5.0]",None
salt,"[{'pouring_line': '2'}, {'pouring_line': '3'}]",07:05,"[6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 13.0, 14.0, 15.0, 16.0, 17.0]",None


In [45]:

schedule = Block('schedule')

def add_block(boiling_type):
    boiling_id = lines_df.at[boiling_type, 'boiling_ids_left'].pop(0)
    boiling_plan = gen_boiling_plan(boiling_plan_df, boiling_id)
    boiling = make_boiling(boiling_plan)

    if not lines_df.at[boiling_type, 'latest_boiling']:
        beg = cast_t(lines_df.at[boiling_type, 'start_time']) - boiling['melting'].beg()
    else:
        beg = lines_df.at[boiling_type, 'latest_boiling'].x1

    dummy_push(schedule, boiling, iter_props=lines_df.at[boiling_type, 'iter_props'], validator=boiling_validator, beg=beg, max_tries=100)

    lines_df.at[boiling_type, 'latest_boiling'] = boiling
    return boiling

In [46]:
for i in range(5):
    add_block('water');

<ipython-input-4-79e6b335079c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boiling_plan['group'] = group_values
<ipython-input-4-79e6b335079c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boiling_plan['group'] = group_values
<ipython-input-4-79e6b335079c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

# Draw boilings

In [47]:
def add_push_with_props(parent, block, **kwargs):
    new_props = block.props.get_all_props()
    new_props.update(kwargs)
    return add_push(parent, block, new_props=new_props)

maker, front, make = new_block_maker('front', default_push_func=dummy_push)

make('stub', h=1, push_func=dummy_push_y)

with make('header', h=1, index_width=3, push_func=dummy_push_y, visible=False):
    make(h=1, size=1, text='График наливов', push_func=add_push)
    for i in range(288):
        cur_time = cast_time(i + cast_t('01:00'))
        if cur_time[-2:] == '00':
            make(t=1 + i, size=1, h=1, text=str(int(cur_time[:2])), color=(218, 150, 148), text_rotation=90, push_func=add_push)
        else:
            make(t=1 + i, size=1, h=1, text=cur_time[-2:], color=(204, 255, 255), text_rotation=90, push_func=add_push)

for i, boiling in enumerate(listify(schedule['boiling'])):
    with make('boiling', push_func=dummy_push_y, block_front_id=f'{i} {boiling.props["pouring_line"]}'):
        make(boiling['pouring'], push_func=add_push_with_props, push_kwargs={'y': 0})
        make(boiling['drenator'], push_func=add_push_with_props, push_kwargs={'y': 1})
        make(boiling['melting']['serving'], push_func=add_push_with_props, push_kwargs={'y': 1})
        make(boiling['melting']['meltings'], push_func=add_push_with_props, push_kwargs={'y': 1})
        
        n_cooling_lines = 4
        cooling_lines = [make('cooling_line', h=1, push_func=dummy_push_y).block for i in range(n_cooling_lines)]
        cur_i = 0
        for cooling_process in listify(boiling['melting']['coolings']['cooling_process']):
            for cooling in cooling_process.iter():
                if cooling.props['class'] in ['cooling', 'salting']:
                    add_push_with_props(cooling_lines[cur_i % n_cooling_lines], cooling)
            cur_i += 1

In [48]:
# merge
# boiling_type = 'water'
# n_cooling_lines = 4
# with make('line', h=2 * (n_cooling_lines + 2) + 1, push_func=dummy_push_y):
#     for i in range(2):
#         with make(f'pouring_line_{i}', h=n_cooling_lines + 1, push_func=dummy_push_y):
#             with make(h=2, push_func=dummy_push_y):
#                 for boiling in [boiling for boiling in listify(schedule['boiling']) if boiling.props['pouring_line'] == str(i)]:
#                     make(boiling['pouring'], push_func=add_push_with_props, push_kwargs={'y': 0})
# #                     make(boiling['drenator'], y=1, push_func=add_push_with_props, push_kwargs={'y': 1})
#                     make(boiling['melting']['serving'], y=1, push_func=add_push_with_props, push_kwargs={'y': 1})
#                     make(boiling['melting']['meltings'], y=1, push_func=add_push_with_props, push_kwargs={'y': 1})
            
#             cooling_lines = [make(f'cooling_{i}', h=1, push_func=dummy_push_y).block for i in range(n_cooling_lines)]

#             cur_i = 0
#             for boiling in [boiling for boiling in listify(schedule['boiling']) if boiling.props['boiling_model'].boiling_type == boiling_type and boiling.props['pouring_line'] == str(i)]:
#                 for cooling_process in listify(boiling['melting']['coolings']['cooling_process']):
#                     for cooling in cooling_process.iter():
#                         if cooling.props['class'] in ['cooling', 'salting']:
#                             add_push_with_props(cooling_lines[cur_i % n_cooling_lines], cooling)
#                     cur_i += 1
#         make('stub', h=1, push_func=dummy_push_y)

In [49]:
style = load_style()
wb = draw_schedule(front, style, init_sheet_func=init_sheet)
wb.save('output.xlsx')
open_file_in_os('output.xlsx') 

# Generate packings

In [221]:
boiling_batch_id = 5
boiling_batch = boiling_plan_df[boiling_plan_df['batch'] == boiling_batch_id]
boiling_batch

,id,boiling,sku,kg,packing_speed,boiling_form_factor,packing_team_id,batch
11,7.0,<Boiling 1>,<SKU 34>,850.0,1020,<BoilingFormFactor 8>,2.0,5
12,8.0,<Boiling 1>,<SKU 14>,800.0,728,<BoilingFormFactor 5>,2.0,5
13,8.0,<Boiling 1>,<SKU 4>,50.0,360,<BoilingFormFactor 1>,2.0,5
14,9.0,<Boiling 1>,<SKU 4>,850.0,360,<BoilingFormFactor 1>,2.0,5
15,10.0,<Boiling 1>,<SKU 4>,850.0,360,<BoilingFormFactor 1>,2.0,5
16,11.0,<Boiling 1>,<SKU 17>,850.0,1020,<BoilingFormFactor 9>,2.0,5
17,13.0,<Boiling 1>,<SKU 35>,850.0,1020,<BoilingFormFactor 9>,2.0,5
18,14.0,<Boiling 1>,<SKU 39>,850.0,1020,<BoilingFormFactor 9>,2.0,5
19,15.0,<Boiling 1>,<SKU 35>,850.0,1020,<BoilingFormFactor 9>,2.0,5


In [222]:
batch_boilings = []
ids = boiling_batch['id']
ids = list(ids)
ids = remove_duplicates(ids)
ids

[7.0, 8.0, 9.0, 10.0, 11.0, 13.0, 14.0, 15.0]

In [223]:
# stupid basic logic without packings
for id in ids:
    boiling_plan = gen_boiling_plan(boiling_plan_df, id)
    batch_boilings.append(make_boiling(boiling_plan))

<ipython-input-4-79e6b335079c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boiling_plan['group'] = group_values
<ipython-input-4-79e6b335079c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boiling_plan['group'] = group_values
<ipython-input-4-79e6b335079c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [224]:
packing_df = boiling_batch[['id', 'packing_team_id', 'boiling_form_factor', 'kg', 'packing_speed']].copy()
packing_df['cur_speed'] = 0
packing_df['left_to_pack'] = packing_df['kg']
packing_df['beg_ts'] = None
packing_df['end_ts'] = None
packing_df['packed'] = 0
packing_df

,id,packing_team_id,boiling_form_factor,kg,packing_speed,cur_speed,left_to_pack,beg_ts,end_ts,packed
11,7.0,2.0,<BoilingFormFactor 8>,850.0,1020,0,850.0,None,None,0
12,8.0,2.0,<BoilingFormFactor 5>,800.0,728,0,800.0,None,None,0
13,8.0,2.0,<BoilingFormFactor 1>,50.0,360,0,50.0,None,None,0
14,9.0,2.0,<BoilingFormFactor 1>,850.0,360,0,850.0,None,None,0
15,10.0,2.0,<BoilingFormFactor 1>,850.0,360,0,850.0,None,None,0
16,11.0,2.0,<BoilingFormFactor 9>,850.0,1020,0,850.0,None,None,0
17,13.0,2.0,<BoilingFormFactor 9>,850.0,1020,0,850.0,None,None,0
18,14.0,2.0,<BoilingFormFactor 9>,850.0,1020,0,850.0,None,None,0
19,15.0,2.0,<BoilingFormFactor 9>,850.0,1020,0,850.0,None,None,0


In [228]:
boiling = batch_boilings[1]

values = []
for melting_process in boiling.iter({'class': 'melting_process'}):
    values.append([melting_process.beg(), 
                   melting_process.beg() + melting_process.props['actual_time_size'], 
                   melting_process.props['boiling_form_factor'], 
                   melting_process.props['boiling_model'].meltings.speed])
melting_process_df = pd.DataFrame(values, columns=['beg_ts', 'end_ts', 'boiling_form_factor', 'speed'])
melting_process_df['beg_ts'] = melting_process_df['beg_ts'] * 60
melting_process_df['end_ts'] = melting_process_df['end_ts'] * 60
melting_process_df

,beg_ts,end_ts,boiling_form_factor,speed
0,3720,6543.529412,<BoilingFormFactor 5>,1020.0
1,4320,4496.470588,<BoilingFormFactor 1>,1020.0


In [229]:
bff_df = dff = pd.DataFrame(index=packing_df['boiling_form_factor'].unique(), columns=['cur_speed', 'cur_kg'])
dff['cur_speed'] = 0
dff['cur_kg'] = 0
dff

,cur_speed,cur_kg
<BoilingFormFactor 8>,0,0
<BoilingFormFactor 5>,0,0
<BoilingFormFactor 1>,0,0
<BoilingFormFactor 9>,0,0


In [230]:
timeline = SortedList([[0, 'start', {}]], key=lambda v: v[0])
for i, row in melting_process_df.iterrows():
    timeline.add([row['beg_ts'], 'melting.start', {'row': i}])
    timeline.add([row['end_ts'], 'melting.finish', {'row': i}])

prev_ts = 0

while timeline:
    cur_ts, cur_topic, cur_data = timeline.pop(0)
    print('Processing time', cur_ts)
    
    # add accumulated meltings
    dff['cur_kg'] += (cur_ts - prev_ts) * dff['cur_speed'] / 3600
    
    # add accumulated packings
    packing_df['packed'] = (cur_ts - prev_ts) * packing_df['cur_speed'] / 3600
    packing_df['left_to_pack'] -= packing_df['packed']
    for i, row in packing_df.iterrows():
        dff.at[row['boiling_form_factor'], 'cur_kg'] -= row['packed'] 
    # null current packing speeds
    packing_df['cur_speed'] = 0
    
    # set finished packings
    packing_df['end_ts'] = np.where(packing_df['end_ts'].isnull() & (packing_df['left_to_pack'].abs() < 1e-2), cur_ts, packing_df['end_ts'])
    
    # set new meltings
    if cur_topic == 'melting.start':
        row = melting_process_df.iloc[cur_data['row']]
        dff.at[row['boiling_form_factor'], 'cur_speed'] = row['speed']
    
    if cur_topic == 'melting.finish':
        row = melting_process_df.iloc[cur_data['row']]
        dff.at[row['boiling_form_factor'], 'cur_speed'] = 0
    
    # set new packings
    
    dff['speed_left'] = dff['cur_speed']
    for team_id, grp in packing_df.groupby('packing_team_id'):
        grp = grp[grp['end_ts'].isnull()]
        
        if len(grp) == 0:
            continue

        cur_packing_row = grp.iloc[0]
        speed_left = dff.at[cur_packing_row['boiling_form_factor'], 'speed_left']
        speed = min(speed_left, cur_packing_row['packing_speed'])
        dff.at[cur_packing_row['boiling_form_factor'], 'speed_left'] -= speed
        packing_df.at[cur_packing_row.name, 'cur_speed'] = speed
    
    packing_df['finish_packing_ts'] = cur_ts + packing_df['left_to_pack'] / packing_df['cur_speed'] * 3600
    packing_df['finish_packing_ts'] = np.where(packing_df['end_ts'].isnull(), packing_df['finish_packing_ts'], None)
    
    packing_checkpoint_ts = packing_df['finish_packing_ts'].min()
    
    if np.isfinite(packing_checkpoint_ts): 
        timeline.add([packing_checkpoint_ts, 'packing_checkpoint', {}])
    
    # start new packings
    packing_df['beg_ts'] = np.where(packing_df['beg_ts'].isnull() & (packing_df['cur_speed'] != 0), cur_ts, packing_df['beg_ts'])
    
    prev_ts = cur_ts
    
    print('Current packing')
    print(display(packing_df))
    print()
    print()
    print('Current dff')
    print(display(dff))
    print()
    print()

Processing time 0
Current packing


,id,packing_team_id,boiling_form_factor,kg,packing_speed,cur_speed,left_to_pack,beg_ts,end_ts,packed,finish_packing_ts
11,7.0,2.0,<BoilingFormFactor 8>,850.0,1020,0,0.0,3720,6720,0.0,None
12,8.0,2.0,<BoilingFormFactor 5>,800.0,728,0,800.0,None,None,0.0,inf
13,8.0,2.0,<BoilingFormFactor 1>,50.0,360,0,50.0,None,None,0.0,inf
14,9.0,2.0,<BoilingFormFactor 1>,850.0,360,0,850.0,None,None,0.0,inf
15,10.0,2.0,<BoilingFormFactor 1>,850.0,360,0,850.0,None,None,0.0,inf
16,11.0,2.0,<BoilingFormFactor 9>,850.0,1020,0,850.0,None,None,0.0,inf
17,13.0,2.0,<BoilingFormFactor 9>,850.0,1020,0,850.0,None,None,0.0,inf
18,14.0,2.0,<BoilingFormFactor 9>,850.0,1020,0,850.0,None,None,0.0,inf
19,15.0,2.0,<BoilingFormFactor 9>,850.0,1020,0,850.0,None,None,0.0,inf


None


Current dff


,cur_speed,cur_kg,speed_left
<BoilingFormFactor 8>,0,0.0,0
<BoilingFormFactor 5>,0,0.0,0
<BoilingFormFactor 1>,0,0.0,0
<BoilingFormFactor 9>,0,0.0,0


None


Processing time 3720
Current packing


...

None


Processing time 7676.043956043956
Current packing


,id,packing_team_id,boiling_form_factor,kg,packing_speed,cur_speed,left_to_pack,beg_ts,end_ts,packed,finish_packing_ts
11,7.0,2.0,<BoilingFormFactor 8>,850.0,1020,0,0.000000,3720,6720,0.0,None
12,8.0,2.0,<BoilingFormFactor 5>,800.0,728,0,229.019608,3720,None,0.0,inf
13,8.0,2.0,<BoilingFormFactor 1>,50.0,360,0,50.000000,None,None,0.0,inf
14,9.0,2.0,<BoilingFormFactor 1>,850.0,360,0,850.000000,None,None,0.0,inf
15,10.0,2.0,<BoilingFormFactor 1>,850.0,360,0,850.000000,None,None,0.0,inf
16,11.0,2.0,<BoilingFormFactor 9>,850.0,1020,0,850.000000,None,None,0.0,inf
17,13.0,2.0,<BoilingFormFactor 9>,850.0,1020,0,850.000000,None,None,0.0,inf
18,14.0,2.0,<BoilingFormFactor 9>,850.0,1020,0,850.000000,None,None,0.0,inf
19,15.0,2.0,<BoilingFormFactor 9>,850.0,1020,0,850.000000,None,None,0.0,inf


None


Current dff


,cur_speed,cur_kg,speed_left
<BoilingFormFactor 8>,0,0.000000,0
<BoilingFormFactor 5>,0,229.019608,0
<BoilingFormFactor 1>,0,50.000000,0
<BoilingFormFactor 9>,0,0.000000,0


None


